# Vaje (napake)

## Programerski miljonar (napake)

Razširite vaš program iz prejšnjih vaj tako, da če pride do napake pri branju ali pisanju v datoteko o tem obvestite uporabnika. Ter če uporabnik vnese število namesto imena ali opisa končajte kviz z napako z opisom `Izgubili ste pravico do kviza!`.

In [ ]:
# Prostor za kodo
import random
import json
import os

# Preverjanje vnosa
def preveri_vnose(vnosi):
    for vnos in vnosi:
        try:
            int(vnos)
        except ValueError:
            pass
        else:
            raise ValueError("Izgubili ste pravico do kviza!")


# Preberi podatke iz datoteke
def preberi(datoteka):
    # Ustvarimo seznam, ki ga bomo uporabili
    kviz = []
    with open(datoteka) as odprto:
        while True:
            prebrano = odprto.readline()
            if not prebrano:
                break

            (kontrola, *niz) = prebrano.strip().split()
            if kontrola == "vprasanje":
                vprasanje = {}
                vprasanje[kontrola] = " ".join(niz)
                # Preberemo odgovore
                odgovori = []
                for i in range(4):
                    (kontrola, *niz) = odprto.readline().strip().split()
                    odgovori.append((kontrola, " ".join(niz)))
                vprasanje["odgovori"] = odgovori
                # Preberemo pravilni odgovor
                (kontrola, *niz) = odprto.readline().strip().split()
                vprasanje[kontrola] = int(niz[0])
                # Dodamo vprašanje v kviz
                kviz.append(vprasanje)
    # Vrnemo kviz
    return kviz

# Preberemo podatke o kvizu
kviz = preberi("datoteke/kviz.txt")
#print(kviz)

# Začetek programa, preberemo podatke
print("Pozdravljeni v kvizu!")
ime = input("Ime tekmovalca:")
opis = input("Opis tekmovalca:")

# Preverimo če sta vnosa celi števili
vnosi = [ime, opis]
preveri_vnose(vnosi)

# Izpišemo pozdrav
print(f"Danes se bo preizkusil {ime}, ki je {opis.lower()}.")

# Naključno premešamo vprašanja
random.shuffle(kviz)

pravilni_odgovori = 0
uspesno = False

for vnos in kviz:
    # Izpišemo vprašanje
    vprasanje = vnos["vprasanje"]
    odgovori = vnos["odgovori"]
    pravilni_odgovor = vnos["pravilni_odgovor"]
    print(f"Vprašanje, ki sledi je: \n\n{vprasanje:^100}")
    print("\nMožni odgovori so: \n")
    for ((crka1, odgovor1), (crka2, odgovor2)) in zip(odgovori[::2], odgovori[1::2]):
        print(f"{crka1} - {odgovor1:.<46}|{crka2:.>45} - {odgovor2}")

    # Odpremo vnos za odgovor
    odgovor = ord(input(f"\nVnesite pravilni odgovor ({[moznost for (moznost, _) in odgovori]})"))

    # Če odgovor ni pravilen se kviz konča
    if odgovor != pravilni_odgovor:
        print(f"\nVaš odgovor {chr(odgovor)} je napačen. Pravilni odgovor je {chr(pravilni_odgovor)}.")
        break
    
    pravilni_odgovori += 1
else:
    uspesno = True
    print("\n\nČestitamo! Uspešno ste rešili kviz.\n\n")

print(f"Pravilno ste odgovorili na {pravilni_odgovori} vprašanj.")

# Sestavimo rezultat
rezultat_tekmovalca = {
    "ime": ime,
    "opis": opis,
    "rezultat": pravilni_odgovori,
    "zmagovalec": uspesno
}

# Izpis v datoteko
pot_do_datoteke = "datoteke/rezultati.json"
# Preberemo oz. preverimo trenutno stanje izpisne datoteke
rezultati_kviza = []
if (os.path.isfile(pot_do_datoteke)):
    with open(pot_do_datoteke) as datoteka:
        rezultati_kviza = json.load(datoteka)

# Dodamo rezultat v seznam slovarjev
rezultati_kviza.append(rezultat_tekmovalca)

# Zapišemo rezulat nazaj v json datoteko
with open(pot_do_datoteke, "w") as datoteka:
    datoteka.write(json.dumps(rezultati_kviza))